# NER4ID at SemEval-2022 Task 2: Named Entity Recognition for Idiomaticity Detection
----------------------------------------------------------------------------------------------------

In [1]:
!git clone https://github.com/babelscape/ner4id

Cloning into 'ner4id'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 56 (delta 20), reused 41 (delta 10), pack-reused 0
Unpacking objects: 100% (56/56), 1.58 MiB | 6.56 MiB/s, done.


## Dependencies

In [2]:
! pip install transformers
! pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os

from tqdm import tqdm

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
 
from pprint import pprint
import random
import numpy as np

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

from transformers import BertTokenizer, BertModel, BertConfig

In [4]:
os.chdir("../")

In [5]:
print(torch.__version__)
torch.cuda.current_device()
torch.cuda.get_device_name(0)

2.0.0+cu118


'Tesla T4'

## Read Data

In [6]:
mode = input("Do you want to train a model from scratch or do you want to evaluate a pretrained model? Possible answers: ['train', 'eval']\n> ")
assert mode in ['train', 'eval']

setting = input("\nIn which setting do you want to train/evaluate your model? Possible answers: ['zero-shot', 'one-shot']\n>")
assert setting in ['zero-shot', 'one-shot']

Do you want to train a model from scratch or do you want to evaluate a pretrained model? Possible answers: ['train', 'eval']
> train

In which setting do you want to train/evaluate your model? Possible answers: ['zero-shot', 'one-shot']
>zero-shot


In [7]:
train_file_zero_shot = "/content/ner4id/data/train_zero_shot.csv" 
train_file_one_shot = "/content/ner4id/data/train_one_shot.csv"

dev_file = "/content/ner4id/data/dev.csv"
dev_file_gold = "/content/ner4id/data/dev_gold.csv"
test_file = "/content/ner4id/data/test.csv"

In [8]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [9]:
!python -m spacy download pt_core_news_sm

2023-04-07 23:49:41.248893: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 9.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [10]:
import spacy
import en_core_web_sm
import pt_core_news_sm
from spacy.cli.download import download as spacy_download

spacy_download("en_core_web_sm")
spacy_tagger_en = spacy.load("en_core_web_sm", exclude=["ner", "parser"])

spacy_download("pt_core_news_sm")
spacy_tagger_pt = spacy.load("pt_core_news_sm", exclude=["ner", "parser"])

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [11]:
import csv

train_data = {}
idx = 0

predictions = []
labels = []

if setting=="zero-shot":
    with open(train_file_zero_shot, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in tqdm(reader):
            
            if len((row[4].strip() + row[5].strip() + row[6].strip()).split(" ")) < 300:
                text = row[4].strip()+ " " + row[5].strip()+ " " + row[6].strip()
            elif len((row[4].strip() + row[5].strip()).split(" ")) < 300:
                text = row[4].strip()+ " " + row[5].strip()
            else:
                text = row[5].strip()
                
            e = row[2].strip()

            train_data[idx] = {
                                "id": row[0],
                                "lang": row[1],
                                "expression": e,
                                "text": row[5].strip(),
                                "idiomatic": True if row[7] == "0" else False
                            }    
            idx += 1


if setting=="one-shot":
    with open(train_file_one_shot, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in tqdm(reader):
            
            if len((row[4].strip() + row[5].strip() + row[6].strip()).split(" ")) < 300:
                text = row[4].strip()+ " " + row[5].strip()+ " " + row[6].strip()
            elif len((row[4].strip() + row[5].strip()).split(" ")) < 300:
                text = row[4].strip()+ " " + row[5].strip()
            else:
                text = row[5].strip()
                
            e = row[2].strip()
                        
            train_data[idx] = {
                                "id": row[0],
                                "lang": row[1],
                                "expression": e,
                                "text": row[5].strip(),
                                "idiomatic": True if row[7] == "0" else False
                            }    
            idx += 1

print(len(train_data)) 

4492it [00:00, 41344.32it/s]

4492


In [12]:
labels_dev = {}

with open(dev_file_gold, newline='',encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        labels_dev[row[0]] = True if row[3] == "0" else False

dev_data = {}
idx = 0

with open(dev_file, newline='',encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:

        if len((row[3].strip() + row[4].strip() + row[5].strip()).split(" ")) < 300:
            text = row[3].strip() + " " + row[4].strip()+ " " + row[5].strip()
        elif len((row[3].strip() + row[4].strip()).split(" ")) < 300:
            text = row[3].strip()+ " " + row[4].strip()
        else:
            text = row[4].strip()
        
        e = row[2].strip()
                
        
        dev_data[idx] = {
                        "id": row[0],
                        "lang": row[1],
                        "expression": e,
                        "text": row[4].strip(),
                        "idiomatic": labels_dev[row[0]]
                        }
                
        idx += 1
        
    
print(len(dev_data)) 

740


In [13]:
test_data = {}
idx = 0

with open(test_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:

        if len((row[3].strip() + row[4].strip() + row[5].strip()).split(" ")) < 300:
            text = row[3].strip()+ " " + row[4].strip()+ " " + row[5].strip()
        elif len((row[3].strip() + row[4].strip()).split(" ")) < 300:
            text = row[3].strip()+ " " + row[4].strip()
        else:
            text = row[4].strip()
        
        e = row[2].strip()

        test_data[idx] = {
                            "id": row[0],
                            "lang": row[1],
                            "expression": e,
                            "text": row[4].strip(),
                            "idiomatic": True #it's just a FAKE label used to maintain the same structure of the dataset entries, 
                                              #it could be also False
                        }
                
        idx += 1
        
    
print(len(test_data)) 

2343


## Fix Random Seed

In [14]:
SEED = 2 #we set a seed for having replicability of results
 
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Load BERT Model

In [15]:
model_name = 'bert-base-multilingual-cased'
 
bert_config = BertConfig.from_pretrained(model_name, output_hidden_states=True)
bert_tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name, config=bert_config)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Dataset

In [16]:
nlp_en = en_core_web_sm.load()
nlp_pt = pt_core_news_sm.load()

class IdiomDataset(Dataset):
    def __init__(self, 
                 dataset, 
                 tokenizer,
                 languages,
                 device="cuda",
                ) -> None:
        
        self.encoded_data = []
    
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.languages = languages
        self.device = device
        self.__init_encoded_data()

 
    def __init_encoded_data(self):
                
        for idx in tqdm(self.dataset):
            id = self.dataset[idx]["id"]
            lang = self.dataset[idx]["lang"]
            e = self.dataset[idx]["expression"]
            label = self.dataset[idx]["idiomatic"]
            context_tmp = self.dataset[idx]["text"]

            if lang == "EN":
                doc = nlp_en(context_tmp)
                ents = [X.text.lower() for X in doc.ents]
            else:
                doc = nlp_pt(context_tmp)
                ents = [X.text.lower() for X in doc.ents]
            
    
            if e in ents:
                e = context_tmp
                context = context_tmp
            else:
                if context_tmp.find(e) != -1:
                    context = context_tmp[:context_tmp.lower().find(e)] + context_tmp[context_tmp.lower().find(e)+len(e):]

                else:    
                    e = context_tmp
                    context = context_tmp                     
                
        
            tokenized_e = torch.tensor(self.tokenize_mention(e, self.tokenizer, True))
            tokenized_context = torch.tensor(self.tokenize_mention(context, self.tokenizer, True))

            if e!="MWE" and lang in self.languages:
                self.encoded_data.append((idx,
                                          e,
                                          context,
                                          tokenized_e,
                                          tokenized_context,
                                          torch.tensor([-1.0]) if label == True else torch.tensor([1.0]),
                                          id, 
                                          lang))

     
    def tokenize_mention(self, sent, tokenizer, special_tokens):
        encoded_sentence = tokenizer.encode(sent, add_special_tokens = special_tokens)
        return encoded_sentence[:500]
    
    def tokenize_description(self, sent, tokenizer, window):
        encoded_sentence = tokenizer.encode(sent, add_special_tokens = True)
        return encoded_sentence

    def __len__(self):
        return len(self.encoded_data)
 
    def __getitem__(self, idx: int):
        return self.encoded_data[idx]

Create the dataset:

In [17]:
train_dataset = IdiomDataset(train_data, bert_tokenizer, languages = ["EN", "PT", "GL"])
dev_dataset = IdiomDataset(dev_data, bert_tokenizer, languages =["EN", "PT", "GL"])
test_dataset = IdiomDataset(test_data, bert_tokenizer, languages = ["EN","PT", "GL"])

print(len(train_dataset))
print(len(dev_dataset))
print(len(test_dataset))

100%|██████████| 2343/2343 [00:27<00:00, 84.44it/s]

4491
739
2342


Create the dataloader:

In [18]:
def collate(elems: tuple) -> tuple:
    ids, e, texts, expressions, contexts, labels, ids, langs = list(zip(*elems))
    
    pad_expressions = pad_sequence(expressions, batch_first=True, padding_value=0)
    pad_contexts = pad_sequence(contexts, batch_first=True, padding_value=0)
    pad_labels = pad_sequence(labels, batch_first=True, padding_value=0)
 
    return ids, e, texts, pad_expressions, pad_contexts, pad_labels.cuda(), ids, langs


train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate)
dev_dataloader = DataLoader(dev_dataset, batch_size=8, shuffle=False, collate_fn=collate)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collate)

print(len(train_dataloader))
print(len(dev_dataloader))
print(len(test_dataloader))

562
93
293


## Dual-Encoder Architecture

In [19]:
class DualEncoder(nn.Module):
    def __init__(self, hparams):
        super(DualEncoder, self).__init__()
        pprint(params)
 
        self.hparams = hparams
 
        self.expression_encoder = BertModel.from_pretrained(model_name, config=bert_config)
        self.context_encoder = BertModel.from_pretrained(model_name, config=bert_config)

        self.cosine_similarity = nn.CosineSimilarity(dim=-1, eps=1e-6)
        
        self.dropout = nn.Dropout(hparams.dropout)

        self.tanh = nn.Tanh()
            
        for param in self.context_encoder.parameters():
            param.requires_grad = False
  

    def forward(self, expression, context, mask1, mask2):
        
        embedding_context = self.context_encoder.forward(context.cuda(), mask2.cuda())[0]
        embedding_context = embedding_context[:,0,:].squeeze(1)

        embedding_expression = self.expression_encoder.forward(expression.cuda(), mask1.cuda())[0]
        embedding_expression = torch.sum(embedding_expression, 1)
            
        similarities = self.cosine_similarity(embedding_expression, embedding_context) 
                        
        return similarities

In [20]:
class HParams():
    dropout = 0.25
    
params = HParams()

Instantiate the model: 

In [21]:
my_model = DualEncoder(params).cuda()
my_model

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.pr

DualEncoder(
  (expression_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

## Trainer

In [22]:
class Trainer():
    def __init__(self,
                model:nn.Module, 
                loss_function,
                optimizer,
                gradient_accumulation_steps):
        
        self.model = model
        self.loss_function = loss_function
        self.optimizer = optimizer
        self.gradient_accumulation_steps = gradient_accumulation_steps
 
    def padding_mask(self, batch):
        padding = torch.ones_like(batch)
        padding[batch == 0] = 0
        padding = padding.type(torch.uint8)
        return padding
 
    def train(self,
            train_dataset:Dataset, 
            valid_dataset:Dataset,
            epochs:int=1,
            patience:int=10,
            modelname=setting):
        
        print("\nTraining...")
 
        record_dev = 0.0
        full_patience = patience
        modelname = modelname
 
        for epoch in range(epochs):
             if patience>0:
                print(" Epoch {:03d}".format(epoch + 1))

                epoch_loss = 0.0
                self.model.train()
                
                count_batches = 0
                self.optimizer.zero_grad()
                
                for ids, e, text, expressions, contexts, labels, id, lang in tqdm(train_dataset):
                    batch_loss = 0.0

                    mask1 = self.padding_mask(expressions)
                    mask2 = self.padding_mask(contexts)
                    
                    similarities = self.model(expressions, contexts, mask1, mask2)

                    labels = labels.view(-1)
                    batch_loss = self.loss_function(similarities, labels)
                    epoch_loss += batch_loss

                    batch_loss.backward()

                    if count_batches % self.gradient_accumulation_steps == 0:
                        self.optimizer.step()
                        self.optimizer.zero_grad()


                avg_epoch_loss = epoch_loss / len(train_dataset)
                print('[E: {:2d}] train loss = {:0.4f}'.format(epoch+1, avg_epoch_loss))

                valid_loss, f1 = self.evaluate(valid_dataset, epoch)

                if f1>record_dev:
                    record_dev = f1
                    torch.save(self.model.state_dict(), "/content/ner4id/checkpoints/"+modelname+".pt")
                    patience = full_patience
                else:
                    patience -= 1
                   
                print('\t[E: {:2d}] valid loss = {:0.4f}, f1-score = {:0.4f}, patience: {:2d}'.format(epoch+1, valid_loss, f1, patience))


        print("...Done!")

        return avg_epoch_loss

    def evaluate(self, valid_dataset, epoch):

        valid_loss = 0.0
        all_predictions = list()
        all_labels = list()
        
        predictions = {}
         
        self.model.eval()
                    
        for ids, e, text, expressions, contexts, labels, id, lang in tqdm(valid_dataset):
            mask1 = self.padding_mask(expressions)
            mask2 = self.padding_mask(contexts)
            
            with torch.no_grad():
                similarities = self.model(expressions, contexts, mask1, mask2)

            labels = labels.view(-1)
            loss = self.loss_function(similarities, labels)
 
            for i in range(len(similarities)):
                if similarities[i]>0:
                    all_predictions.append(1)
                else:
                    all_predictions.append(0)

                all_labels.append(1 if labels[i].item()==1 else 0)
            
            valid_loss += loss
            
            for i in range(len(similarities)):
                if similarities[i]>0:
                    predictions[ids[i]] = 1

                else:
                    predictions[ids[i]] = 0

        f1 = f1_score(all_labels, all_predictions, average= 'macro')
        print(classification_report(all_labels, all_predictions, digits=4))
        
        return valid_loss / len(valid_dataset), f1

Instantiate the trainer:

In [23]:
trainer = Trainer(model = my_model,
                    loss_function = nn.MSELoss(),
                    optimizer = optim.Adam(my_model.parameters(), lr=0.00001),
                    gradient_accumulation_steps=1)

Train the system:

In [24]:
if mode == "train":
    trainer.train(train_dataloader, dev_dataloader, epochs = 1000, patience = 5)


Training...
 Epoch 001


100%|██████████| 562/562 [01:24<00:00,  6.67it/s]


[E:  1] train loss = 0.4616


100%|██████████| 93/93 [00:05<00:00, 18.58it/s]


              precision    recall  f1-score   support

           0     0.6286    0.9018    0.7408       336
           1     0.8716    0.5558    0.6788       403

    accuracy                         0.7131       739
   macro avg     0.7501    0.7288    0.7098       739
weighted avg     0.7611    0.7131    0.7070       739

	[E:  1] valid loss = 0.9194, f1-score = 0.7098, patience:  5
 Epoch 002


...

              precision    recall  f1-score   support

           0     0.6427    0.7976    0.7118       336
           1     0.7888    0.6303    0.7007       403

    accuracy                         0.7064       739
   macro avg     0.7158    0.7139    0.7063       739
weighted avg     0.7224    0.7064    0.7057       739

	[E: 13] valid loss = 0.9883, f1-score = 0.7063, patience:  1
 Epoch 014


100%|██████████| 562/562 [01:24<00:00,  6.64it/s]


[E: 14] train loss = 0.1604


100%|██████████| 93/93 [00:05<00:00, 18.08it/s]


              precision    recall  f1-score   support

           0     0.6374    0.8423    0.7256       336
           1     0.8203    0.6005    0.6934       403

    accuracy                         0.7104       739
   macro avg     0.7289    0.7214    0.7095       739
weighted avg     0.7372    0.7104    0.7081       739

	[E: 14] valid loss = 0.9648, f1-score = 0.7095, patience:  0
...Done!


## Predict the test set

Load the best model checkpoint:

In [25]:
my_model.load_state_dict(torch.load(f"/content/ner4id/checkpoints/{setting}.pt"))

<All keys matched successfully>

and then use the model to predict the test set:

In [26]:
import os
folder_name = "predictions"
par_dir = "/content/ner4id"
path = os.path.join(par_dir, folder_name)
os.makedirs(path)

In [27]:
def padding_mask(batch):
        padding = torch.ones_like(batch)
        padding[batch == 0] = 0
        padding = padding.type(torch.uint8)
        return padding

def predict(model, test_dataset):

    predictions = {}
        
    model.eval()
                
    for ids, e, text, expressions, contexts, labels, id, lang in tqdm(test_dataset):
        mask1 = padding_mask(expressions)
        mask2 = padding_mask(contexts)
        
        with torch.no_grad():
            similarities = model(expressions, contexts, mask1, mask2)

        labels = labels.view(-1)

        for i in range(len(similarities)):
            if similarities[i]>0:
                predictions[ids[i]] = 1

            else:
                predictions[ids[i]] = 0


    predictions["ID"] = "Label"

    if setting == "zero-shot":
        with open("/content/ner4id/data/test_submission_format.csv") as csvfile:
            with open("/content/ner4id/predictions/task2_subtaska.csv", "w") as out:
                reader = csv.reader(csvfile, delimiter=',')
                for row in reader:
                    if row[2] == "zero_shot" or row[0] == "ID":
                        out.write(row[0] + "," + row[1] + "," + row[2] + "," + str(predictions[row[0]]) + "\n")
                    else:
                        out.write(row[0] + "," + row[1] + "," + row[2] + "," + "" + "\n")
    else:
        with open("/content/ner4id/data/test_submission_format.csv") as csvfile:
            with open("/content/ner4id/predictions/task2_subtaska.csv", "w") as out:
                reader = csv.reader(csvfile, delimiter=',')
                for row in reader:
                    if row[2] == "one_shot" or row[0] == "ID":
                        out.write(row[0] + "," + row[1] + "," + row[2] + "," + str(predictions[row[0]]) + "\n")
                    else:
                        out.write(row[0] + "," + row[1] + "," + row[2] + "," + "" + "\n")    


predict(my_model, test_dataloader)

100%|██████████| 293/293 [00:15<00:00, 19.15it/s]


The output file (saved in the /predictions folder) contains predictions in the standard format specified by the competition rules. You can now upload the output file on CodaLab and see the scores obtained by the system on the test set.